# Times obtained during automatic analysis test (3 Orgs - 1 P. Node per Org)

### Imports

In [1]:
import asyncio
import websockets
import json
import time
import hashlib
import os
import requests
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

### Global variables

In [2]:
API_ENDPOINT = "http://35.195.161.115:3001/api/"
WS_ENDPOINT = "ws://35.195.161.115:3000/api/"
NS = "ertis.uma.nuclear"

### Main function

In [3]:
async def eventListener(DEBUG=False):
    print("EVENT LISTENER STARTED")
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred)
    async with websockets.connect(WS_ENDPOINT) as websocket:
        if DEBUG:
            print("Waiting for new event...")
        cont = 0
        time_list = []
        while cont<100:
            aux = await websocket.recv()
            if DEBUG:
                print("NEW EVENT")
            aux2 = json.loads(aux)
            eventId = aux2['eventId']
            acqId = aux2['acqId']
            filename = aux2['filename']
            hash_value = aux2['hash']
            if DEBUG:
                print(f"Event ID: {eventId}")
                print(f"Acquisition ID: {acqId}")
                print(f"Filename: {filename}")
                print(f"Hash value: {hash_value}")

            #Waiting a little bit for file upload to repository by acquisitor process
            time.sleep(10)

            #Download file from repository
            downloadFileFromRepository(filename)

            #Compare downloaded file hash value with stored hash value
            if checkHashSHA256(filename, hash_value):
                if DEBUG:
                    print("Valid hash")

                #Get file content
                acqData = getFileContent(filename)

                #Delete local file
                deleteLocalFile(filename)

                #Send transaction
                elapsed_time = addAutomaticAnalysis(acqId, acqData)
                time_list.append(elapsed_time)
                if DEBUG:
                    print(f"Elapsed time adding automatic analysis: {elapsed_time}\r\n")
            else:
                if DEBUG:
                    print("Not valid hash\r\n")
            cont = cont+1
        minimum = min(time_list)
        average = sum(time_list)/len(time_list)
        maximum = max(time_list)
        print(f"Fastest automatic analysis: {minimum}")
        print(f"Average time while executing automatic analysis: {average}")
        print(f"Slowest automatic analysis: {maximum}")

### Auxiliar functions

In [4]:
def downloadFileFromRepository(filename):
    bucket = storage.bucket("hyperledger-jte.appspot.com")

    blob = bucket.get_blob(filename)
    blob.download_to_filename(filename)

def checkHashSHA256(filename, hashStored):
    hashValue = sha256(filename)
    return hashValue == hashStored

def sha256(fname):
    hash_sha256 = hashlib.sha256()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

def getFileContent(filename):
    content = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            content.append(int(lines[i]))
    return content

def deleteLocalFile(filename):
    if(os.path.exists(filename)):
        os.remove(filename)
    else:
        print(f"File {filename} does not exist")

def addAutomaticAnalysis(acqId, acqData, DEBUG=False):
    resource_url = f"{API_ENDPOINT}{NS}.AddAutomaticAnalysis"
    data = {
        "analysisId": acqId,
        "acqId": acqId,
        "acqData": acqData
    }
    start_time = time.time()
    r = requests.post(resource_url, data=data)
    elapsed_time = time.time() - start_time
    if DEBUG:
        print(f"Elapsed time: {elapsed_time}")
        print(f"Response status code: {r.status_code}")
        print(r.json())
    return elapsed_time

### Main function execution

In [5]:
asyncio.get_event_loop().run_until_complete(eventListener(True))

RuntimeError: This event loop is already running

EVENT LISTENER STARTED
Waiting for new event...
